# __THE TELCO CHURN CHALLENGE FOR REXEL__

## 1. Introduction

The objective of this challenge is to prevent customer to stop using TELCO Inc phoning services.

There are many reasons why customers may churn. It's crucial to detect those customers before they leave.

One of the most effective way to achive that goal is to use the data.

Based on historical data, we are going to detect customers who may leave and suggest actions that can avoid the leaving.

## 2. The data

We have 2 datasets to achieve the challenge. _The training dataset_, will be use to train, test and evaluate machine learning models. The validation dataset is for the final submission of the challenge.

In [12]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/training.csv', na_values=[" "])

In [13]:
print("The dataset shape: ", data.shape)

The dataset shape:  (11981, 19)


The data we have for this challenge has __11981__ rows and __19__ columns (or variables). Let's visualize the first 6 rows of the data:

In [14]:
data.head().T

,0,1,2,3,4
CUSTOMER_ID,C100000,C100001,C100006,C100008,C100010
COLLEGE,zero,one,zero,zero,one
DATA,660,317.647,208.696,265.018,440
INCOME,19995,31477,66742,40864,43321.5
OVERCHARGE,0,155,0,183,200
LEFTOVER,0,15,13,0,0
HOUSE,897338,393396,937197,986430,394622
LESSTHAN600k,False,True,False,False,True
CHILD,4,0,4,3,2
JOB_CLASS,3,1,2,3,3


In [15]:
data.dtypes

CUSTOMER_ID                     object
COLLEGE                         object
DATA                           float64
INCOME                         float64
OVERCHARGE                       int64
LEFTOVER                         int64
HOUSE                          float64
LESSTHAN600k                    object
CHILD                            int64
JOB_CLASS                        int64
REVENUE                        float64
HANDSET_PRICE                    int64
OVER_15MINS_CALLS_PER_MONTH      int64
TIME_CLIENT                    float64
AVERAGE_CALL_DURATION            int64
REPORTED_SATISFACTION           object
REPORTED_USAGE_LEVEL            object
CONSIDERING_CHANGE_OF_PLAN      object
CHURNED                         object
dtype: object

### 2.1 The data description

* CUSTOMER_ID: A unique customer identifier (categorical)
* COLLEGE: (one or zero), is the customer college educated ? (categorical)
* DATA: Monthly data consumption in Mo (numerical)
* INCOME: Annual salary of the client (numerical)
* OVERCHARGE:Average overcharge per year (numerical)
* LEFTOVER: Average number of leftover minutes per month (numerical)
* HOUSE: Estimated value of the house (numerical)
* LESSTHAN600k: Is the value of the house smaller than 600K ? (catagorical)
* CHILD: The number of children (numerical)
* JOB_CLASS: Self reported type of job (categorical)
* REVENUE: Annual phone bill (numerical)
* HANDSET_PRICE: The price of the handset (phone) (numerical)
* OVER_15MINS_CALLS_PER_MONTH: Average number of long calls (more than 15 minutes) (numerical)
* TIME_CLIENT: The tenure in year (numerical)
* AVERAGE_CALL_DURATION: The average duration of a call (numerical)
* REPORTED_SATISFACTION: The reported level of satisfaction (categorical)
* REPORTED_USAGE_LEVEL: The self reported usage level (categorical)
* CONSIDERING_CHANGE_OF_PLAN: Self reported consideration whether to change operator (categorical)
* CHURNED: Did the customer stay or leave. This is the class (categorical)

Let's convert each variable in the appropriate data type:

In [16]:
data['CUSTOMER_ID'] = pd.Categorical(data['CUSTOMER_ID'])
data['COLLEGE'] = pd.Categorical(data['COLLEGE'])
data['LESSTHAN600k'] = pd.Categorical(data['LESSTHAN600k'])
data['JOB_CLASS'] = pd.Categorical(data['JOB_CLASS'])
data['REPORTED_SATISFACTION'] = pd.Categorical(data['REPORTED_SATISFACTION'])
data['REPORTED_USAGE_LEVEL'] = pd.Categorical(data['REPORTED_USAGE_LEVEL'])
data['CONSIDERING_CHANGE_OF_PLAN'] = pd.Categorical(data['CONSIDERING_CHANGE_OF_PLAN'])
data['CHURNED'] = pd.Categorical(data['CHURNED'])

### 2.2 The data quality

Before starting any analysis, it's important to guarantee the quality of the data. Especially, we are going to check if there are missing values:

In [17]:
data.isna().sum()

CUSTOMER_ID                      0
COLLEGE                          0
DATA                             0
INCOME                           0
OVERCHARGE                       0
LEFTOVER                         0
HOUSE                          635
LESSTHAN600k                   635
CHILD                            0
JOB_CLASS                        0
REVENUE                          0
HANDSET_PRICE                    0
OVER_15MINS_CALLS_PER_MONTH      0
TIME_CLIENT                      0
AVERAGE_CALL_DURATION            0
REPORTED_SATISFACTION            0
REPORTED_USAGE_LEVEL             0
CONSIDERING_CHANGE_OF_PLAN       0
CHURNED                          0
dtype: int64

There are 2 variables with missing values: HOUSE (the house value) and the LESSTHAN600K (is the house value smaller or higher tha 600K ?). 
 

__What is the type of the missing values ?__

In [18]:
# We retain only rows with missing values for the variable HOUSE
dataNa = data[data['HOUSE'].isna()]
lessthan600k = dataNa['LESSTHAN600k']

# The percentage of missing values in the column LESSTHAN600K
100*lessthan600k.isna().sum()/lessthan600k.shape[0]

100.0

The variable __LESSTHAN600K__ has missing values because the house were not estimated.

Let's evaluate the percentage of missing values in the whole data set:

In [19]:
100*lessthan600k.shape[0]/data.shape[0]

5.3000584258409145

There are only __5.3%__ of rows with a missing value.

We can then drop all the rows with missing values without introducing a biais in the data.

In [20]:
data = data.dropna()

In [21]:
data.isna().sum()

CUSTOMER_ID                    0
COLLEGE                        0
DATA                           0
INCOME                         0
OVERCHARGE                     0
LEFTOVER                       0
HOUSE                          0
LESSTHAN600k                   0
CHILD                          0
JOB_CLASS                      0
REVENUE                        0
HANDSET_PRICE                  0
OVER_15MINS_CALLS_PER_MONTH    0
TIME_CLIENT                    0
AVERAGE_CALL_DURATION          0
REPORTED_SATISFACTION          0
REPORTED_USAGE_LEVEL           0
CONSIDERING_CHANGE_OF_PLAN     0
CHURNED                        0
dtype: int64

## 3. Data exploratory analysis

Before the modeling, it's important to dive deep inside the data, to highlith the link or correlation between varaiables.
This intend to ease the choice of a machine learning model family.

### 3.1 Numerical variables exploration

In [22]:
dataNumeric = data.select_dtypes(exclude='category')
dataNumeric.describe().T

,count,mean,std,min,25%,50%,75%,max
DATA,11346.0,503.381766,457.877611,0.0,181.442686,360.564416,684.563758,6600.0
INCOME,11346.0,47554.643266,22301.121475,10032.5,29344.000000,44874.000000,65027.625000,105355.5
OVERCHARGE,11346.0,80.228627,84.648846,-2.0,0.000000,56.000000,171.000000,298.0
LEFTOVER,11346.0,23.857218,26.720440,0.0,0.000000,15.000000,41.000000,89.0
HOUSE,11346.0,506521.681650,256808.319571,150015.0,268511.250000,469775.500000,722464.250000,999996.0
CHILD,11346.0,2.069364,1.251756,0.0,1.000000,2.000000,3.000000,6.0
REVENUE,11346.0,155.497180,47.873219,50.0,121.000000,149.000000,182.000000,521.0
HANDSET_PRICE,11346.0,382.952142,210.254384,130.0,216.250000,322.000000,513.750000,899.0
OVER_15MINS_CALLS_PER_MONTH,11346.0,7.525560,8.755728,0.0,1.000000,4.000000,13.000000,29.0
TIME_CLIENT,11346.0,3.080407,1.651883,0.3,1.900000,2.700000,3.800000,21.0


are there correlations ?

In [23]:
numericCorr = dataNumeric.corr(method='pearson')
numericCorr

,DATA,INCOME,OVERCHARGE,LEFTOVER,HOUSE,CHILD,REVENUE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,TIME_CLIENT,AVERAGE_CALL_DURATION
DATA,1.000000,-0.129805,-0.362800,-0.266912,-0.040929,-0.256063,0.005983,0.010887,-0.275918,0.542037,0.247626
INCOME,-0.129805,1.000000,0.368734,0.011422,-0.019709,0.001667,0.003557,0.665961,0.286259,0.002187,-0.017347
OVERCHARGE,-0.362800,0.368734,1.000000,0.010360,0.035223,0.015281,0.013880,-0.018056,0.774034,-0.007353,0.006899
LEFTOVER,-0.266912,0.011422,0.010360,1.000000,0.015579,-0.000094,-0.007507,0.011341,-0.001680,0.002115,-0.657271
HOUSE,-0.040929,-0.019709,0.035223,0.015579,1.000000,0.008648,-0.001388,-0.028208,0.032863,0.006234,-0.014851
CHILD,-0.256063,0.001667,0.015281,-0.000094,0.008648,1.000000,-0.002738,-0.004719,0.016546,-0.001009,0.001007
REVENUE,0.005983,0.003557,0.013880,-0.007507,-0.001388,-0.002738,1.000000,-0.005575,0.006824,-0.007596,-0.003081
HANDSET_PRICE,0.010887,0.665961,-0.018056,0.011341,-0.028208,-0.004719,-0.005575,1.000000,-0.008995,-0.004889,-0.022832
OVER_15MINS_CALLS_PER_MONTH,-0.275918,0.286259,0.774034,-0.001680,0.032863,0.016546,0.006824,-0.008995,1.000000,-0.006970,0.018990
TIME_CLIENT,0.542037,0.002187,-0.007353,0.002115,0.006234,-0.001009,-0.007596,-0.004889,-0.006970,1.000000,0.003497


We can draw some observations according to the table above:
There is a strong correlation between:
* __OVERCHARGE__ and __OVER_15MINS_CALLS_PER_MONTH__ (average number of long calls)
* __LEFTOVER__ and __AVERAGE_CALL_DURATION__
* ...

Let's display all